In [1]:
# this folder is used to explore quantities
import re
import pandas as pd
import numpy as np


cookbook = pd.read_csv('../data/cookbook_df.csv')
tj_inventory = pd.read_csv("../data/trader_joes_products_v2.csv")

In [14]:
cookbook

,title,category,quantity_text,url,image_url,serves,time,name
0,Aromatic Garlic Ginger Rice,Appetizers & Sides,4 tablespoons,https://www.traderjoes.com/home/recipes/aromat...,https://www.traderjoes.com/content/dam/trjo/co...,Serves 4,Time 20 mins,butter
1,Aromatic Garlic Ginger Rice,Appetizers & Sides,2 tablespoons,https://www.traderjoes.com/home/recipes/aromat...,https://www.traderjoes.com/content/dam/trjo/co...,Serves 4,Time 20 mins,garlic
2,Aromatic Garlic Ginger Rice,Appetizers & Sides,1 tablespoon,https://www.traderjoes.com/home/recipes/aromat...,https://www.traderjoes.com/content/dam/trjo/co...,Serves 4,Time 20 mins,ginger
3,Aromatic Garlic Ginger Rice,Appetizers & Sides,1 cup,https://www.traderjoes.com/home/recipes/aromat...,https://www.traderjoes.com/content/dam/trjo/co...,Serves 4,Time 20 mins,jasmine rice
4,Aromatic Garlic Ginger Rice,Appetizers & Sides,2 cups,https://www.traderjoes.com/home/recipes/aromat...,https://www.traderjoes.com/content/dam/trjo/co...,Serves 4,Time 20 mins,hot water
...,...,...,...,...,...,...,...,...
3696,Mini Caprese,Appetizers & Sides,1 package,https://www.traderjoes.com/home/recipes/mini-c...,https://www.traderjoes.com/content/dam/trjo/re...,Serves 6,Time 10 mins,basil
3697,Mini Caprese,Appetizers & Sides,1 package,https://www.traderjoes.com/home/recipes/mini-c...,https://www.traderjoes.com/content/dam/trjo/re...,Serves 6,Time 10 mins,ciliegine mozzarella
3698,Mini Caprese,Appetizers & Sides,1 bottle,https://www.traderjoes.com/home/recipes/mini-c...,https://www.traderjoes.com/content/dam/trjo/re...,Serves 6,Time 10 mins,balsamic vinaigrette
3699,Mini Caprese,Appetizers & Sides,1/4 teaspoon,https://www.traderjoes.com/home/recipes/mini-c...,https://www.traderjoes.com/content/dam/trjo/re...,Serves 6,Time 10 mins,sea salt


In [2]:
def split_quantity(qty):
    if pd.isna(qty) or not isinstance(qty, str) or qty.strip() == "":
        return pd.Series([np.nan, np.nan])

    qty = qty.lower().strip()

    # Match numeric part (integer, float, or fraction) + unit
    match = re.match(r"([\d¼½¾⅓⅔⅛⅜⅝⅞\s\/\.]+)\s*([a-zA-Z]+.*)?", qty)
    if match:
        amount = match.group(1).strip()
        unit = match.group(2).strip() if match.group(2) else np.nan
        return pd.Series([amount, unit])
    else:
        return pd.Series([np.nan, np.nan])

In [15]:
ingre= cookbook[['title','name', 'quantity_text']].copy()
ingre[["amount", "unit"]] = ingre["quantity_text"].apply(split_quantity)

In [16]:
ingre = ingre.dropna().reset_index(drop=True)

fractions_map = {
    "½": "1/2",
    "¼": "1/4",
    "¾": "3/4",
    "⅓": "1/3",
    "⅔": "2/3",
    "⅛": "1/8",
    "⅜": "3/8",
    "⅝": "5/8",
    "⅞": "7/8"
}

def normalize_fraction(text):
    if isinstance(text, str):
        for sym, rep in fractions_map.items():
            text = text.replace(sym, rep)
        return text
    return text

ingre["amount"] = ingre["amount"].apply(normalize_fraction)


In [17]:
ingre.head()

,title,name,quantity_text,amount,unit
0,Aromatic Garlic Ginger Rice,butter,4 tablespoons,4,tablespoons
1,Aromatic Garlic Ginger Rice,garlic,2 tablespoons,2,tablespoons
2,Aromatic Garlic Ginger Rice,ginger,1 tablespoon,1,tablespoon
3,Aromatic Garlic Ginger Rice,jasmine rice,1 cup,1,cup
4,Aromatic Garlic Ginger Rice,hot water,2 cups,2,cups


In [18]:
wood = ingre[ingre['unit'] == 'wooden']
wood


,title,name,quantity_text,amount,unit
1019,Fruity Frozen Yogurt Pops,craft sticks,6 wooden,6,wooden


In [7]:
store = tj_inventory[['product_name', 'unit']].copy()

In [8]:
def parse_store_unit(u):
    if not isinstance(u, str):
        return pd.Series([None, None])

    # Extract numbers and unit (e.g., "12", "oz" from "/12 Oz")
    match = re.match(r"\/?([\d\.]+)\s*([a-zA-Z\s]+)", u)
    if match:
        qty = float(match.group(1))
        unit = match.group(2).strip().lower()
        return pd.Series([qty, unit])
    else:
        return pd.Series([None, None])

store[["store_qty", "store_unit"]] = store["unit"].apply(parse_store_unit)

In [11]:
store.head()

,product_name,unit,store_qty,store_unit
0,Herbed Dinner Rolls,/12 Oz,12.00,oz
1,Pumpkin & Spice Brioche Style Liège Waffles,/11.64 Oz,11.64,oz
2,Glazed Pumpkin Pie Spice Donut Holes,/6 Oz,6.00,oz
3,Pumpkin Spice Mini Sheet Cake,/18 Oz,18.00,oz
4,Sliced Apple Cinnamon Sourdough Bread,/17.63 Oz,17.63,oz


In [9]:
store['store_unit'].unique()

array(['oz', 'lb', 'fl oz', 'doz', 'pint', 'qt', 'each'], dtype=object)